In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets
import numpy as np
import cv2
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Load CIFAR-10 dataset

In [ ]:
def preprocess(images):
    images = images.astype('float32') / 255.0
    return images

In [ ]:
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    keypoints_list = []
    descriptors_list = []
    for image in images:
        image = (image * 255).astype(np.uint8)  # Convert image to uint8
        keypoints, descriptors = sift.detectAndCompute(image, None)
        if descriptors is not None and len(keypoints) > 0:  # Ensure valid descriptors
            descriptors_list.append(descriptors)
            keypoints_list.append(keypoints)
    return keypoints_list, descriptors_list

In [ ]:
def bow_representation(keypoints, descriptors, kmeans):
    bow_representation = np.zeros((len(keypoints), kmeans.n_clusters), dtype=np.float32)
    for i in range(len(keypoints)):
        if descriptors[i] is not None:
            words = kmeans.predict(descriptors[i])
            for word in words:
                bow_representation[i, word] += 1
    return bow_representation

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Preprocessing

train_images = preprocess(train_images)
test_images = preprocess(test_images)

# Feature extraction using SIFT


train_keypoints, train_descriptors = extract_sift_features(train_images)
test_keypoints, test_descriptors = extract_sift_features(test_images)

# Flatten descriptors
train_descriptors_flat = np.vstack(train_descriptors)
test_descriptors_flat = np.vstack(test_descriptors)


In [ ]:
kmeans = KMeans(n_clusters=12, random_state=42)
kmeans.fit(train_descriptors_flat)

# Generate Bag-of-Visual Words representation


train_bow = bow_representation(train_keypoints, train_descriptors, kmeans)
test_bow = bow_representation(test_keypoints, test_descriptors, kmeans)

# Scale features
scaler = StandardScaler()
train_bow_scaled = scaler.fit_transform(train_bow)
test_bow_scaled = scaler.transform(test_bow)

In [ ]:
# import optuna


# def objective(trial):
#     C = trial.suggest_loguniform('C', 1e-10, 1e10)
#     # kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
#     svm = SVC(C=C, kernel='linear')
    
#     # Fit the classifier to the training data
#     svm.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

#     # Make predictions on the training and test data
#     train_predictions = svm.predict(train_bow_scaled)
#     test_predictions = svm.predict(test_bow_scaled)

#     # Calculate accuracy
#     train_accuracy = accuracy_score(train_labels[:len(train_bow)], train_predictions)
#     test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

#     # Return the test accuracy as the objective value to maximize
#     return test_accuracy

# # Create a study object and optimize the objective function
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# Get the best hyperparameters
# best_params = study.best_params
best_svm = SVC()

# Fit the classifier with the best hyperparameters
best_svm.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

# Make predictions on the test data using the best classifier
test_predictions = best_svm.predict(test_bow_scaled)

# silhouette = silhouette_score(test_bow_scaled, test_labels)
# print("Silhouette Score:", silhouette)
# Calculate accuracy
test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

# print("Best hyperparameters:", best_params)
# print("Test Accuracy with best model:", test_accuracy)/


In [ ]:
# def objective(trial):
#     C = trial.suggest_loguniform('C', 1e-10, 1e10)
#     max_iter = trial.suggest_int('max_iter', 100, 1000)
    
#     # Initialize the classifier with the suggested hyperparameters
#     clf = LogisticRegression(C=C, max_iter=max_iter, random_state=42)
    
#     # Fit the classifier to the training data
#     clf.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

#     # Make predictions on the training and test data
#     train_predictions = clf.predict(train_bow_scaled)
#     test_predictions = clf.predict(test_bow_scaled)

#     # Calculate accuracy
#     train_accuracy = accuracy_score(train_labels[:len(train_bow)], train_predictions)
#     test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

#     # Return the test accuracy as the objective value to maximize
#     return test_accuracy

# # Create a study object and optimize the objective function
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# # Get the best hyperparameters
# best_params = study.best_params
best_lr = LogisticRegression(random_state=42)

# Fit the classifier with the best hyperparameters
best_lr.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

# Make predictions on the test data using the best classifier
test_predictions = best_lr.predict(test_bow_scaled)

# Calculate accuracy
test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

print("Best hyperparameters:", best_params)
print("Test Accuracy with best model:", test_accuracy)


In [ ]:
# from sklearn.metrics import silhouette_score


# def objective(trial):
#     n_neighbors = trial.suggest_int('n_neighbors', 1, 100)
#     knn_clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    
#     # Fit the classifier to the training data
#     knn_clf.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

#     # Make predictions on the training and test data
#     train_predictions = knn_clf.predict(train_bow_scaled)
#     test_predictions = knn_clf.predict(test_bow_scaled)

#     # Calculate accuracy
#     train_accuracy = accuracy_score(train_labels[:len(train_bow)], train_predictions)
#     test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

#     # Return the test accuracy as the objective value to maximize
#     return test_accuracy

# # Create a study object and optimize the objective function
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=10)

# # Get the best hyperparameters
# best_params = study.best_params
best_knn = KNeighborsClassifier()

# Fit the classifier with the best hyperparameters
best_knn.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

# Make predictions on the test data using the best classifier
test_predictions = best_knn.predict(test_bow_scaled)

# Calculate accuracy
test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

print("Best hyperparameters:", best_params)
print("Test Accuracy with best model:", test_accuracy)